# PIC-SURE API use-case: Phenome-Wide analysis on COPDgene data

This is a tutorial notebook, aimed to be quickly up and running with the python PIC-SURE API. It covers the main functionalities of the API.

## PIC-SURE python API 
### What is PIC-SURE? 

<!--img src="./img/PIC-SURE_logo.png" width= "360px"> -->

Databases exposed through PIC-SURE API encompass a wide heterogeneity of architectures and data organizations underneath. PIC-SURE hide this complexity and expose the different databases in the same format, allowing researchers to focus on the analysis and medical insights, thus easing the process of reproducible sciences.

### More about PIC-SURE
PIC-SURE stands for Patient-centered Information Commons: Standardized Unification of Research Elements. The API is available in two different programming languages, python and R, allowing investigators to query databases in the same way using any of those languages.

PIC-SURE is a large project from which the R/python PIC-SURE API is only a brick. Among other things, PIC-SURE also offers a graphical user interface, allowing research scientist to get quick knowledge about variables and data available for a specific data source.

The python API is actively developed by the Avillach-Lab at Harvard Medical School.

GitHub repo:
* https://github.com/hms-dbmi/pic-sure-python-adapter-hpds
* https://github.com/hms-dbmi/pic-sure-python-client



 -------   

# Environment set-up

**Before running this notebook, please be sure to review the HPDS_connection.ipynb notebook. It contains explanation about how to get a security token, mandatory to access the databases.**

### Pre-requisite
- python 3.6 or later (although earlier versions of Python 3 must work too)
- pip: python package manager, already available in most system with a python interpreter installed ([pip installation instructions](https://pip.pypa.io/en/stable/installing/))

### IPython magic command

Those two lines of code below do load the `autoreload` IPython extension. Although not necessary to execute the rest of the Notebook, it does enable to reload every dependency each time python code is executed, thus enabling to take into account changes in external file imported into this Notebook (e.g. user defined function stored in separate file), without having to manually reload libraries. Turns out very handy when developing interactively. More about [IPython Magic commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html).

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Useful to estimate execution time of the Notebook, see end of this file
from datetime import datetime
then = datetime.now()

### Installation of required python packages

Using the pip package manager, we install the packages listed in the `requirements.txt` file.

In [ ]:
!cat requirements.txt

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

Import all the external dependencies, as well as user-defined functions stored in the `python_lib` folder

In [ ]:
import json
from pprint import pprint

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from scipy import stats

import PicSureHpdsLib
import PicSureClient

from python_lib.utils import get_multiIndex_variablesDict, get_dic_renaming_vars, match_dummies_to_varNames, joining_variablesDict_onCol
from python_lib.HPDS_connection_manager import tokenManager

In [ ]:
print("NB: This Jupyter Notebook has been written using PIC-SURE API following versions:\n- PicSureClient: 0.1.0\n- PicSureHpdsLib: 1.1.0\n")
print("The PIC-SURE API libraries versions you've been downloading are: \n- PicSureClient: {0}\n- PicSureHpdsLib: {1}".format(PicSureClient.__version__, PicSureHpdsLib.__version__))

##### Set up the options for displaying tables and plots in this Notebook

In [ ]:
# Pandas DataFrame display options
pd.set_option("max.rows", 435)

# Matplotlib parameters options
fig_size = plt.rcParams["figure.figsize"]
 
# Prints: [8.0, 6.0]
fig_size[0] = 14
fig_size[1] = 8
plt.rcParams["figure.figsize"] = fig_size

font = {'weight' : 'bold',
        'size'   : 12}

plt.rc('font', **font)

### Connecting to a PIC-SURE network

In [ ]:
PICSURE_network_URL = "https://biodatacatalyst.integration.hms.harvard.edu/picsure"
resource_id = "02e23f52-f354-4e8b-992c-d37c8b9ba140"
token_file = "token.txt"

In [ ]:
token = tokenManager(token_file).get_token()

In [ ]:
client = PicSureClient.Client()
connection = client.connect(PICSURE_network_URL, token)
adapter = PicSureHpdsLib.Adapter(connection)
resource = adapter.useResource(resource_id)

Finally, we created an object called `resource`, which is an instance of the `PicSureHpdsLib.Adapter()` class. It is connected to the specific resources we indicated, namely COPDGene hosted database in our case. 

**This `resource` object is actually the only one we will need to proceed with our analysis thereafter**.

#### Getting help with the PIC-SURE python API

Each object exposed by the PicSureHpdsLib library got a `help()` method. Calling it will print a helper message about it. 

In [ ]:
resource.help()

For instance, this output tells us that this `resource` object got 2 methods, and it gives insights about their function. 

## Using the *variables dictionnary*

Once connection to the desired resource has been established, we first need to get a quick grasp of which variables are available in the database. To this end, we will use the `dictionary` method of the `resource` object.

A `dictionary` instance offers the possibility to retrieve matching records according to a specific term, or to retrieve information about all available variables, using the `find()` method. For instance, looking for variables containing the term `COPD`: 

In [ ]:
dictionary = resource.dictionary()
lookup = dictionary.find("COPD")

Subsequently, objects created by the `dictionary.find` exposes the search result using 4 different methods: `.count()`, `.keys()`, `.entries()`, and `.DataFrame()`. 

In [ ]:
pprint({"Count": lookup.count(), 
        "Keys": lookup.keys(),
        "Entries": lookup.entries()})

In [ ]:
lookup.DataFrame()

**`.DataFrame()` appears as the most useful method for an end-user**. 

* Various criteria exposed in the dictionary (patientCount, variable type ...) can be subsequently used as selection criteria for variable selection.
* Row names of the DataFrame, representing actual variables names, can be used in the query, instead of typing directly the name of the variable in the source code.

Variable names, as currently implemented in the API, aren't very practical to use.
1. Very long
2. Presence of backslashes that prevent from copy-pasting. 

However, using the dictionary to select variables can definitely help to deal with this pitfall. Hence, one handy way to proceed is to retrieve the whole dictionary in the form of a pandas DataFrame, as below:

In [ ]:
plain_variablesDict = resource.dictionary().find("\\Genetic Epidemiology of COPD (COPDGene)\\Subject Phenotype\\6MinWalk\\Walk symptoms: back pain\\").DataFrame()

Indeed, using the find function without arguments return every entries, as stated by the help documentation.

In [ ]:
resource.dictionary().help()

In [ ]:
plain_variablesDict

The dictionary currently returned by the API provide various information about the variables, such as:
- observationCount: number of entries with non-null value
- categorical: type of the variables, True if categorical, False if continuous/numerical
- min/max: only provided for non-categorical variables
- HpdsDataType: 'phenotypes' or 'genotypes'. Currently COPDGene instance only contains 'phenotypes' variables

#### Variable dictionary + pandas multiIndex

Though helpful, we can use a simple user-defined function (`get_multiIndex_variablesDict`) to add a little more information and ease dealing with variables names. It takes advantage of pandas MultiIndex functionality [see pandas official documentation on this topic](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html).

Although not an official feature of the API, such functionality to quickly scan an select groups of related variables may be integrated at some point.

Printing the 'multiIndexed' variable Dictionary allows to quickly see the tree-like organisation of the variables. Moreover, original and simplified variable names are now stored respectively in the "varName" and "simplified_varName" columns.

In [ ]:
variablesDict = get_multiIndex_variablesDict(plain_variablesDict)

In [ ]:
variablesDict

In [ ]:
# Now that we have seen how our entire dictionnary looked, we limit the number of lines to be displayed for the future outputs
pd.set_option("max.rows", 50)

Below is a simple example to illustrate the ease of use a multiIndex Dictionary. Let's say we are interested in every variables pertaining to the "Medical history" and "Medication history" subcategories.

In [ ]:
idx = pd.IndexSlice
mask_medication = variablesDict.index.get_level_values(2) == "Medication History"
mask_diseases = variablesDict.index.get_level_values(2) == "Medical History"
medication_history_variables = variablesDict.loc[mask_diseases | mask_medication,:]
medication_history_variables

Although pretty simple, it can be easily combined with other filters to quickly select necessary variables.

## Querying the COPDGene HPDS database

Beside from the dictionary, the second cornerstone of the API is the `query` object. It is the entering point to retrieve data from the resource.

In [ ]:
query = resource.query()

The most simple usage of the query object is passing a variable name through the `select` method.

The `query.select().add()` method accept variable names as string or list of strings as argument.

In [ ]:
query.select().add("\\DCC Harmonized data set\\03 - Baseline common covariates\\Body mass index calculated at baseline.\\")
query.getResultsDataFrame()
query.getCount()

#### Selecting variables

There is many different methods provided by the API: `select`, `require`, `anyof`, `filter`, and each one of those methods can be combined with `add` and `delete` to create queries. Moreover, different results can be returned for a single query: `getCount`, `getResults` ... Information about each one of those functions can be found using `help()`.

However, **a simple straightforward workflow is to simply select the desired variables using the Dictionary, enter their names using `query.select().add()`, and then retrieve the data using `query.getResultsDataFrame()` method**.

Let's say we are interested in the variables pertaining to the 'Respiratory disease form' category, and that we only want the categorical ones, with at least 4000 non-null values. One simple way to process is:

In [ ]:
query = resource.query()
mask_cat = variablesDict["categorical"] == True
mask_count = variablesDict["observationCount"] > 4000
varnames = variablesDict.loc[idx[:, :, :, "Cardio-Cerebro-Vascular"],:].loc[mask_cat & mask_count, "varName"]
query.select().add(varnames)
query_result = query.getResultsDataFrame()
query_result.head()